# Approach

These are the key components of the WiFi co-processor:

* **Software on ESP32:** MicroPython. No need to reinvent the wheel. And it's guaranteed MicroPython compatible.
* **Interface:** We'll use remote procedure calls (RPC) to invoke functions on the ESP32 (server) from the client.
* **Data Encoding:** [MessagePack](https://msgpack.org/) is used to encode Python objects sent to or received from the ESP32.
* **Communications Channel:** [UART](https://en.wikipedia.org/wiki/Universal_asynchronous_receiver-transmitter) - it's widely available on virtually all microcontrollers, requires only two wires, and is symmetrical: both endpoints can communicate at any time without complex handshaking. Most microcontrollers support baud rates up to and in excess of 1Mbps, sufficient for communication not to be the bottleneck in this application. 


## Software on ESP32

Nothing special here. Get whatever ESP32 you wish to use - a full development board with USB or a module for as little as two Dollars. Then flash a MicroPython interpreter.

## MessagePack

Follow these [instructions](custom_c.ipynb) to compile a custom version of MicroPython with support for MessagePack.

## UART

## RPC

Python is ideally suited for encapsulating calls for remote execution. Let's cook up a simple class to check out how this could work. 

It's standard Python - you can run this on any (Micro)Python interpreter.

In [1]:
%connect esp32

Connected to esp32 @ serial:///dev/ttyUSB0


In [1]:
class Demo:
    
    def __init__(self, name):
        self._name = name
        
    def add(self, a, b):
        return "{}: {} + {} = {}".format(self._name, a, b, a+b)
    
    @property
    def name(self):
        return self._name
    
    def __str__(self):
        return "Demo {}".format(self._name)
    
d = Demo("rpc test")
print("ADD ", d.add(5, 7))
print("NAME", d.name)
print("STR ", d)

ADD  rpc test: 5 + 7 = 12
NAME rpc test
STR  Demo rpc test


To perform these operations remotely, we need a handle to the remote *Demo* instance *d*. Let's create a *Proxy* for this purpose: the proxy exists on the client, the object (*d* in the example) on the server. 

For development we can run both parts in the same interpreter.

In [ ]:
class Proxy:

    def __init__(self, ext_type: msgpack.ExtType):
        # _Proxy is a reference (msgpack.ExtType) to an object on the remote
        assert isinstance(ext_type, msgpack.ExtType)
        self._ext_type = ext_type
        super().__init__(self.__del__)

    @property
    def ext_type(self):
        return self._ext_type

    def __getattr__(self, name: str):
        # Note:
        # We return an accessor function rather than a reference to the
        # actual object (method, property) on the server.
        # This avoids an extra rpc call but has the disadvantage that
        # properties cannot accessed the standard way.
        # (_Proxy.x just returns the accessor method, not the property value).
        def method(*args, **kwargs):
            return _send(("cm", self._ext_type, name, args, kwargs))
        return method

    def get_(self, name: str):
        # get object property
        return _send(("gp", self._ext_type, name))

    def set_(self, name: str, value):
        # set object property
        return _send(("sp", self._ext_type, name, value))

    def readinto(self, buffer):
        global _stream
        _clear_rx_buffer()
        msgpack.pack(("ri", self._ext_type, len(buffer)), _stream)
        _ready_to_read()
        # get actual number of bytes read OR handle error (if any)
        sz = msgpack.unpack(_stream, ext_hook=_ext_handler, use_list=False)
        # read data; not all versions of readinto support length argument
        mv = memoryview(buffer)
        # print("urpc client.readinto [sz]")
        _stream.readinto(mv[:sz])
        # server sends an extra None
        assert msgpack.unpack(_stream) == None, "readinto expects terminating 'None'"
        return sz

    # pre-allocate (heap locked in __del__)
    DEL = [ "_d", None ]

    def __del__(self):
        # reclaim object on remote
        global _stream
        _Proxy.DEL[1] = self._ext_type.data
        msgpack.pack(self.DEL, _stream)
        return msgpack.unpack(_stream)  # read response (None)

    def __str__(self):
        return _send(("_s", self._ext_type))
    
    def __eq__(self, other):
        return id_(self) == id_(other)



Update local MicroPython branch:

In [1]:
%%bash

cd ~/micropython
git checkout master
git pull
git merge master

Already on 'master'
Your branch is up to date with 'origin/master'.
From https://github.com/micropython/micropython
   e3291e180..71722c84c  master     -> origin/master
Updating e3291e180..71722c84c
Fast-forward
 .github/workflows/ports_unix.yml                   |  12 +
 docs/esp32/quickref.rst                            |   2 +-
 docs/esp8266/quickref.rst                          |   6 +-
 docs/library/machine.I2C.rst                       |  38 +-
 docs/library/machine.I2S.rst                       |   2 +-
 docs/library/machine.SPI.rst                       |  14 +-
 docs/library/pyb.I2C.rst                           |  54 +-
 docs/library/pyb.SPI.rst                           |  20 +-
 docs/library/uasyncio.rst                          |   8 +
 docs/library/utime.rst                             |   9 +
 docs/pyboard/quickref.rst                          |  12 +-
 docs/pyboard/tutorial/amp_skin.rst                 |   2 +-
 docs/pyboard/tutorial/lcd_skin.rst                 |   6 +

Find port:

In [ ]:
%info

Compile and flash:

In [1]:
%%service esp-idf

cd ~/micropython/ports/esp32
# make submodules
# make clean 
make BOARD=GENERIC_OTA \
     USER_C_MODULES=../../../../micropython_modules/micropython.cmake \
     PORT=/dev/ttyUSB0 deploy

setting up IDF ...
idf.py -D MICROPY_BOARD=GENERIC_OTA -B build-GENERIC_OTA  -DUSER_C_MODULES=../../../../micropython_modules/micropython.cmake -p /dev/ttyUSB0 -b 460800 flash
[1/206] cd /home/iot/micropython/ports/esp32/build-GENERIC_OTA/esp-idf/main && echo -n
[2/203] Performing build step for 'bootloader'
ninja: no work to do.
[3/201] Generating ../../genhdr/qstr.split
[4/201] Generating ../../genhdr/qstrdefs.collected.h
QSTR updated
[5/201] Generating ../../genhdr/qstrdefs.preprocessed.h
[6/201] Generating ../../genhdr/qstrdefs.generated.h
[7/201] Generating ../../frozen_content.c
[8/201] Building C object esp-idf/main/CMakeFiles/__idf_main.dir/home/iot/micropython/py/frozenmod.c.obj
[9/201] Building C object esp-idf/main/CMakeFiles/__idf_main.dir/home/iot/micropython/py/argcheck.c.obj
[10/201] Building C object esp-idf/main/CMakeFiles/__idf_main.dir/home/iot/micropython/py/map.c.obj
[11/201] Building C object esp-idf/main/CMakeFiles/__idf_main.dir/home/iot/micropython/py/bc.c.obj


In [1]:
try:
    from finaliser_proxy import FinaliserProxy
except ImportError:
    # CPython compatibility (and ports that do not implement this feature)
    print("no finaliser proxy ...")
    class FinaliserProxy:
        def __init__(self, cb):
            pass
    
import gc

class FP(FinaliserProxy):
    def __init__(self, desc):
        self.desc = desc
        super().__init__(self.__del__)
    def __del__(self):
        print("finalise", self.desc)

for i in range(20):
    f = FP("obj {}".format(i))
    if i % 4 == 0:
        print("--------- collect ...", i)
        gc.collect()

gc.collect()

print("DONE")

no finaliser proxy ...
--------- collect ... 0
--------- collect ... 4
--------- collect ... 8
--------- collect ... 12
--------- collect ... 16
DONE


## UART